In [2]:
# import pickle

# # Specify the file path where the label encoder is saved
# file_path = 'label_encoder.pkl'

# # Load the label encoder from the file
# with open(file_path, 'rb') as file:
#     label_encoder = pickle.load(file)


In [3]:
import pandas as pd
df = pd.read_json('dataset.json')

In [4]:
df

,externalStatus,internalStatus
0,PORT OUT,Port Out
1,TERMINAL IN,Inbound Terminal
2,PORT IN,Port In
3,Vessel departure from first POL (Vessel name :...,Departure
4,Vessel arrival at final POD (Vessel name : TIA...,Arrival
...,...,...
1217,Import Loaded on Rail,Loaded on Vessel
1218,Full Transshipment Loaded,Loaded on Vessel
1219,Full Transshipment Loaded,Loaded on Vessel
1220,Export Loaded on Vessel,Loaded on Vessel


In [5]:
import re
def remove_special_characters(text):
    return re.sub(r'\b(?:[A-Z]+|\d+\w*|\w*\d+)\b\s*|(\s*\(.*?\)\s*)|[^\w\s]', '', text).strip()

df['preocessed_ext'] = df['externalStatus'].apply(remove_special_characters)

In [6]:
df["preocessed_ext"].unique()

array(['', 'Vessel departure from first', 'Vessel arrival at final',
       'Gate in', 'Load on', 'Discharge', 'Gate out', 'Load', 'On rail',
       'Off rail', 'Gate out Empty', 'Empty Container Release to Shipper',
       'Gate In to Outbound Terminal', 'Loaded on  at Port of Loading',
       'Departure from Port of Loading', 'Arrival at Port of Discharging',
       'Unloaded from  at Port of Discharging',
       'Empty Container Returned from Customer',
       'Empty Container Door Arrival', 'Gate In to Outbound',
       'Outbound Rail Loading', 'Outbound Rail Departure',
       'Outbound Rail Arrival', 'Outbound Rail Unloading',
       'Gate Out from Outbound', 'Loaded', 'Unloaded',
       'Actual time of departure', 'Expected time of departure',
       'Empty to shipper', 'Vessel Departure', 'Vessel Arrival',
       'Loaded on board', 'Full Load on rail for export',
       'Train arrival for export', 'Import Unloaded from Rail',
       'Import Loaded on Rail', 'Full Transshipment 

In [7]:
len(df["preocessed_ext"].unique())

52

In [8]:
empty_rows = df[df['preocessed_ext'] == '']
print(empty_rows["externalStatus"].unique())


['PORT OUT' 'TERMINAL IN' 'PORT IN' 'DEPARTCU' 'TOLL PLAZA CROSSED'
 'CFS OUT' 'CFS IN' 'TERMINAL OUT' 'GATE OUT']


In [9]:
def fill_empty(row):
    if row['preocessed_ext'] == '':
        return row['externalStatus']
    else:
        return row['preocessed_ext']
df['preocessed_ext'] = df.apply(fill_empty, axis=1)

In [10]:
len(df["preocessed_ext"].unique())

60

In [11]:
df["internalStatus"].unique()

array(['Port Out', 'Inbound Terminal', 'Port In', 'Departure', 'Arrival',
       'Gate In', 'Loaded on Vessel', 'Gate Out', 'On Rail', 'Off Rail',
       'Empty Return', 'In-transit', 'Outbound Terminal',
       'Empty Container Released', 'Unloaded on Vessel'], dtype=object)

In [12]:
len(df["internalStatus"].unique())

15

In [13]:
df.to_csv('dataset_clean.csv', index=False)

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

label_encoder_1 = LabelEncoder()
label_encoder_2 = LabelEncoder()
df['externalStatus_encoded'] = label_encoder_1.fit_transform(df['preocessed_ext'])
df['internalStatus_encoded'] = label_encoder_2.fit_transform(df['internalStatus'])

X = df['externalStatus_encoded'].values
y = df['internalStatus_encoded'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Dense(128, activation='relu', input_shape=(1,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(df['internalStatus'].unique()), activation="softmax")
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, verbose=1)

test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy}")


d:\SNU2\sem6\.venv\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.3090 - loss: 2.4645
Epoch 2/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3448 - loss: 2.2630
Epoch 3/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3759 - loss: 2.2105
Epoch 4/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4367 - loss: 2.1550
Epoch 5/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4833 - loss: 2.0496
Epoch 6/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4904 - loss: 1.9022
Epoch 7/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5013 - loss: 1.7820
Epoch 8/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4939 - loss: 1.7915
Epoch 9/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4734 - loss: 1.8043
Epoch 10/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4625 - loss: 1.8368
Epoch 11/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5013 - loss: 1.7106
Epoch 12/100
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy:

In [31]:
# import pickle
with open('tensornn.weights.h5', 'wb') as file:
    model.save_weights('tensornn.weights.h5')

In [32]:
len(df['internalStatus'].unique())

15

In [15]:
from sklearn.metrics import confusion_matrix,classification_report

In [16]:
print(classification_report(y_test, np.argmax(model.predict(X_test), axis=1)))


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
              precision    recall  f1-score   support

           0       0.48      1.00      0.65        12
           1       1.00      0.97      0.98        60
           2       0.72      1.00      0.84        13
           3       0.00      0.00      0.00        13
           4       0.87      1.00      0.93        27
           5       0.78      1.00      0.88        29
           6       0.00      0.00      0.00         2
           7       1.00      1.00      1.00         3
           8       0.93      1.00      0.96        53
           9       0.00      0.00      0.00         7
          10       0.33      1.00      0.50         4
          11       0.00      0.00      0.00         5
          12       1.00      1.00      1.00         4
          13       0.00      0.00      0.00         8
          14       0.00      0.00      0.00         5

    accuracy                           0.83       245
   macro avg       0.47      0.60      0.52

d:\SNU2\sem6\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\SNU2\sem6\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\SNU2\sem6\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, random_state=0)
clf.fit(X_train.reshape(-1, 1), y_train)
clf.score(X_test.reshape(-1, 1), y_test)

0.9959183673469387

In [18]:
X_test

array([29,  6, 48, 29, 27, 13, 27, 31, 25, 13, 47, 33,  6, 16, 10, 33,  6,
       49, 18,  6, 36, 40, 41, 22, 28, 29, 28, 28, 33, 28,  6, 27, 41,  6,
       15,  9, 41, 18,  1, 33, 48, 27,  0, 33, 48, 33, 41, 47,  0, 42, 41,
       27, 33, 28,  4,  0, 29,  8, 36, 27,  6, 10, 52, 31, 28,  6, 22, 33,
       49, 16,  6,  6, 43, 16, 27, 28, 27,  8, 28, 14, 32, 28, 10, 10, 33,
       28, 27,  0,  6, 27,  6,  6,  2, 16, 29,  2, 56, 33,  6,  8, 33, 52,
        6, 33, 28, 48, 29, 29, 22, 27, 10, 27, 33, 42,  6, 48, 35, 27, 27,
       28, 28, 27, 27,  4,  6,  6, 36,  9, 27, 33,  9, 28, 28, 33, 33, 27,
       10, 27, 16, 22,  6, 13, 36, 36,  6, 25,  4,  6, 41, 47,  9, 28, 33,
       27, 55,  6, 33, 48,  1,  6, 29, 33,  4, 37, 33, 28,  0,  6, 28, 28,
       27,  0,  6, 28, 49, 41, 19, 28, 30,  6,  6, 18,  6, 33,  0, 27, 22,
        6, 10, 47, 27,  0, 25, 27, 18, 22, 10, 48, 33, 55, 33, 42,  0, 48,
       27, 36, 33,  4, 25,  0,  6, 18,  9, 28, 37, 18,  2, 56, 27, 33, 33,
       28,  6, 57,  5, 42

In [19]:
print(classification_report(y_test, clf.predict(X_test.reshape(-1, 1))))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        12
           1       1.00      1.00      1.00        60
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00        13
           4       1.00      1.00      1.00        27
           5       1.00      1.00      1.00        29
           6       1.00      1.00      1.00         2
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00        53
           9       1.00      1.00      1.00         7
          10       0.80      1.00      0.89         4
          11       1.00      0.80      0.89         5
          12       1.00      1.00      1.00         4
          13       1.00      1.00      1.00         8
          14       1.00      1.00      1.00         5

    accuracy                           1.00       245
   macro avg       0.99      0.99      0.99       245
weighted avg       1.00   

A random forest classifier works better in this case than the neural network

In [20]:
import pickle
with open('random_forest_model.pkl', 'wb') as file:
    pickle.dump(clf, file)

In [21]:
import pickle

# Specify the file path where you want to save the label encoder
file_path = 'label_encoder1.pkl'

# Save the label encoder to the file
with open(file_path, 'wb') as file:
    pickle.dump(label_encoder_1, file)


In [22]:
data = "Gate out, Empty"

In [24]:
def pre_process(text):
    if text in ['PORT OUT', 'TERMINAL IN', 'PORT IN', 'DEPARTCU', 'TOLL PLAZA CROSSED', 'CFS OUT', 'CFS IN', 'TERMINAL OUT', 'GATE OUT']:
        return text
    else:
        return re.sub(r'\b(?:[A-Z]+|\d+\w*|\w*\d+)\b\s*|(\s*\(.*?\)\s*)|[^\w\s]', '', text).strip()

In [25]:
input_string = label_encoder_1.transform([pre_process(data)])

In [26]:
input_string

array([29])

In [27]:
np.argmax(model.predict([input_string]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step


5

In [28]:
str(label_encoder_2.inverse_transform([np.argmax(model.predict([input_string]))]))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step


"['Gate Out']"

In [ ]:
label_encoder_2.classes_

array(['Arrival', 'Departure', 'Empty Container Released', 'Empty Return',
       'Gate In', 'Gate Out', 'In-transit', 'Inbound Terminal',
       'Loaded on Vessel', 'Off Rail', 'On Rail', 'Outbound Terminal',
       'Port In', 'Port Out', 'Unloaded on Vessel'], dtype=object)

In [ ]:
label_encoder_1.classes_

array(['Actual time of departure', 'Arrival at Port of Discharging',
       'CFS IN', 'CFS OUT', 'DEPARTCU', 'Departure from Port of Loading',
       'Discharge', 'Empty Container Door Arrival',
       'Empty Container Release to Shipper',
       'Empty Container Returned from Customer', 'Empty to Shipper',
       'Empty to shipper', 'Estimated Time of Arrival',
       'Expected Arrival Place of Delivery', 'Expected Arrival Update',
       'Expected Arrival at Dest Ramp', 'Expected time of departure',
       'Export Loaded on Rail', 'Export Loaded on Vessel',
       'Export Rail Departure', 'Export Unloaded from Rail',
       'Full Load on rail for export', 'Full Transshipment Loaded',
       'GATE OUT', 'Gate In to Outbound', 'Gate In to Outbound Terminal',
       'Gate Out from Outbound', 'Gate in', 'Gate out', 'Gate out Empty',
       'Import Loaded on Rail', 'Import Unloaded from Rail', 'Load',
       'Load on', 'Loaded', 'Loaded at', 'Loaded at First',
       'Loaded on  at Port o

In [46]:
label_encoder_2.inverse_transform([np.argmax(model.predict([input_string]))])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


array(['Gate Out'], dtype=object)

In [47]:
# str(label_encoder_2.inverse_transform(model.predict([input_string])))

In [39]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(1,)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(15, activation="softmax")
])
checkpoint_path = './tensornn.weights.h5'
model.load_weights(checkpoint_path)

In [41]:
input_string

array([29])

In [54]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(X_train.reshape(-1, 1), y_train)
clf.score(X_test.reshape(-1, 1), y_test)

0.8122448979591836

In [55]:
import pickle
with open('naive_bayes.pkl', 'wb') as file:
    pickle.dump(clf, file)

In [57]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 128)            │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 15)             │           495 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,087 (43.31 KB)

 Trainable params: 11,087 (43.31 KB)

 Non-trainable params: 0 (0.00 B)

In [58]:
print(classification_report(y_test, clf.predict(X_test.reshape(-1, 1))))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        12
           1       0.76      0.98      0.86        60
           2       0.67      0.77      0.71        13
           3       0.00      0.00      0.00        13
           4       1.00      1.00      1.00        27
           5       0.78      1.00      0.88        29
           6       1.00      1.00      1.00         2
           7       0.00      0.00      0.00         3
           8       0.85      0.89      0.87        53
           9       1.00      1.00      1.00         7
          10       1.00      1.00      1.00         4
          11       0.00      0.00      0.00         5
          12       1.00      1.00      1.00         4
          13       1.00      1.00      1.00         8
          14       0.25      0.40      0.31         5

    accuracy                           0.81       245
   macro avg       0.62      0.67      0.64       245
weighted avg       0.72   

d:\SNU2\sem6\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\SNU2\sem6\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\SNU2\sem6\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
